# 10개 클래스 테스트 (Google Cloud Storage, GCS)

## 정현님이 올려주신 이미지와 같이 데이터 개수 Top 10의 이미지만 먼저 분류해 보자!
    1. issue : 1위 데이터의 개수와 10위 데이터 개수 차이도 너무 크다. 이것이 문제가 되는지도 알아야한다.
      MNIST로 테스트
    2. Issue가 문제가 된다면 해결방법은? 이미지. 개수 밸런싱을 한다.
    3. 간단한 모델로 일단 transfer learning을 해본다.(VGG, Inception, etc)

### 분석결과

#### 1. issue : 1위 데이터의 개수와 10위 데이터 개수 차이도 너무 크다. 이것이 문제가 되는지도 알아야한다.

* 사용한 dataset : MNIST
* train size : 60,000
* test size : 10,000
* label size : 10

|샘플링 유무|클래스별 샘플링 확률|테스트셋 정확도|
|--------|---------------|--------|
|무||9830/10000 (98%)|
|유|각 클래스별로 동일한 확률로 샘플링되도록 셋팅|9839/10000 (98%)|
|유|[0.9, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]|9354/10000 (94%)|
|유|[0.99, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]|6528/10000 (65%)|
|유|[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]|980/10000 (10%)|
|유|[0.24707459, 0.2461469 , 0.1149304 , 0.09066322, 0.0651395, 0.05471404, 0.04666915, 0.04526535, 0.0452359 , 0.04416096]|9767/10000 (98%)|

* 레이블 당 데이터의 비율이 다를때 문제가 되는것 확인
* 마지막 결과는 Kaggle문제 Top10 레이블의 데이터 비율을 그대로 반영하였음
* 결과는 언뜻좋아보이나, 비율이 작은 레이블의 정확도가 떨어질것임으로 간단하게 처리하려면 밸런싱 필요

#### 2. Issue가 문제가 된다면 해결방법은? 이미지. 개수 밸런싱을 한다.
* 문제가 되는것으로 확인 -> 개수를 밸런싱해야함 또는 Imbalanced dataset을 처리하는 기법 분석 필요

#### 3. 간단한 모델로 일단 transfer learning을 해본다.(VGG, Inception, etc)
* Transferlearning 테스트 필요
* 장기적으로는 의령님이 제시해주신것 처럼 다른 기법 분석이 필요


## 1.issue : 1위 데이터의 개수와 10위 데이터 개수 차이도 너무 크다. 이것이 문제가 되는지도 알아야한다.

### 모델 셋팅

In [49]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

In [50]:
!pip3 install torchvision

    100% |████████████████████████████████| 5.9MB 230kB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [51]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

    100% |████████████████████████████████| 5.6MB 236kB/s 
  Found existing installation: Pillow 5.0.0
    Uninstalling Pillow-5.0.0:
      Successfully uninstalled Pillow-5.0.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
# load library
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

class argument():
  def __init__(self):
    pass

args = argument()
args.batch_size = 64
args.test_batch_size = 1000
args.epochs = 10
args.lr = 0.01
args.momentum = 0.5
args.no_cuda = False
args.seed = 1
args.log_interval = 10

# Training settings
args.cuda = True

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

# load dataset
kwargs = {'num_workers': 4, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [0]:
# MNIST CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if args.cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### MNIST 테스트 (기본셋팅)

In [17]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Processing...
Done!
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.399207
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.324624
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.290561
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.270582
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.244259
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.240479
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.183806
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.133017
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.059968
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.012864
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.974352
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.779724
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.803475
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.609995
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.809938
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.601794
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.490459
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.388518
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.451829
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.306679
Train Epo

Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.532794
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.301033
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.435763
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.303979
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.318449
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.458063
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.163945
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.461048
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.437912
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.328404
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.432514
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.364290
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.508250
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.362440
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.511227
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.414815
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.193067
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.535276
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.281813
Train Epoch: 2 [13440/60000 (22%)]	Loss: 0.595524
Train Epoch: 2 [14080

Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.320664
Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.223816
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.304468
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.298083
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.447941
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.281302
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.283003
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.319610
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.401618
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.245683
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.254357
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.172447
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.283569
Train Epoch: 3 [13440/60000 (22%)]	Loss: 0.216359
Train Epoch: 3 [14080/60000 (23%)]	Loss: 0.343991
Train Epoch: 3 [14720/60000 (25%)]	Loss: 0.314845
Train Epoch: 3 [15360/60000 (26%)]	Loss: 0.253369
Train Epoch: 3 [16000/60000 (27%)]	Loss: 0.220691
Train Epoch: 3 [16640/60000 (28%)]	Loss: 0.277275
Train Epoch: 3 [17280/60000 (29%)]	Loss: 0.323316
Train Epo

Train Epoch: 4 [10880/60000 (18%)]	Loss: 0.324244
Train Epoch: 4 [11520/60000 (19%)]	Loss: 0.203278
Train Epoch: 4 [12160/60000 (20%)]	Loss: 0.341055
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.159170
Train Epoch: 4 [13440/60000 (22%)]	Loss: 0.210178
Train Epoch: 4 [14080/60000 (23%)]	Loss: 0.223575
Train Epoch: 4 [14720/60000 (25%)]	Loss: 0.532719
Train Epoch: 4 [15360/60000 (26%)]	Loss: 0.135656
Train Epoch: 4 [16000/60000 (27%)]	Loss: 0.176355
Train Epoch: 4 [16640/60000 (28%)]	Loss: 0.287399
Train Epoch: 4 [17280/60000 (29%)]	Loss: 0.314677
Train Epoch: 4 [17920/60000 (30%)]	Loss: 0.309314
Train Epoch: 4 [18560/60000 (31%)]	Loss: 0.267266
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.172872
Train Epoch: 4 [19840/60000 (33%)]	Loss: 0.252593
Train Epoch: 4 [20480/60000 (34%)]	Loss: 0.238737
Train Epoch: 4 [21120/60000 (35%)]	Loss: 0.212186
Train Epoch: 4 [21760/60000 (36%)]	Loss: 0.272675
Train Epoch: 4 [22400/60000 (37%)]	Loss: 0.142775
Train Epoch: 4 [23040/60000 (38%)]	Loss: 0.367090


Train Epoch: 5 [14080/60000 (23%)]	Loss: 0.222069
Train Epoch: 5 [14720/60000 (25%)]	Loss: 0.211490
Train Epoch: 5 [15360/60000 (26%)]	Loss: 0.263095
Train Epoch: 5 [16000/60000 (27%)]	Loss: 0.253182
Train Epoch: 5 [16640/60000 (28%)]	Loss: 0.159045
Train Epoch: 5 [17280/60000 (29%)]	Loss: 0.306547
Train Epoch: 5 [17920/60000 (30%)]	Loss: 0.366244
Train Epoch: 5 [18560/60000 (31%)]	Loss: 0.161492
Train Epoch: 5 [19200/60000 (32%)]	Loss: 0.204015
Train Epoch: 5 [19840/60000 (33%)]	Loss: 0.278266
Train Epoch: 5 [20480/60000 (34%)]	Loss: 0.289616
Train Epoch: 5 [21120/60000 (35%)]	Loss: 0.085125
Train Epoch: 5 [21760/60000 (36%)]	Loss: 0.111342
Train Epoch: 5 [22400/60000 (37%)]	Loss: 0.150732
Train Epoch: 5 [23040/60000 (38%)]	Loss: 0.149163
Train Epoch: 5 [23680/60000 (39%)]	Loss: 0.249176
Train Epoch: 5 [24320/60000 (41%)]	Loss: 0.176876
Train Epoch: 5 [24960/60000 (42%)]	Loss: 0.205306
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.348645
Train Epoch: 5 [26240/60000 (44%)]	Loss: 0.184858


Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.444145
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.287574
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.176236
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.256376
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.199366
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.198291
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.128507
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.319550
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.112695
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.371763
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.122329
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.248045
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.323675
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.208882
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.194422
Train Epoch: 6 [28160/60000 (47%)]	Loss: 0.166007
Train Epoch: 6 [28800/60000 (48%)]	Loss: 0.379686
Train Epoch: 6 [29440/60000 (49%)]	Loss: 0.124358
Train Epoch: 6 [30080/60000 (50%)]	Loss: 0.302440
Train Epoch: 6 [30720/60000 (51%)]	Loss: 0.164639


Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.179671
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.310513
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.105301
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.093333
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.177452
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.174057
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.379910
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.277351
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.296235
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.102941
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.178715
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.411943
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.166461
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.259052
Train Epoch: 7 [24960/60000 (42%)]	Loss: 0.090835
Train Epoch: 7 [25600/60000 (43%)]	Loss: 0.221911
Train Epoch: 7 [26240/60000 (44%)]	Loss: 0.182248
Train Epoch: 7 [26880/60000 (45%)]	Loss: 0.146269
Train Epoch: 7 [27520/60000 (46%)]	Loss: 0.068788
Train Epoch: 7 [28160/60000 (47%)]	Loss: 0.302078


Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.372753
Train Epoch: 8 [13440/60000 (22%)]	Loss: 0.160678
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.147717
Train Epoch: 8 [14720/60000 (25%)]	Loss: 0.353984
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.134539
Train Epoch: 8 [16000/60000 (27%)]	Loss: 0.172091
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.089058
Train Epoch: 8 [17280/60000 (29%)]	Loss: 0.175412
Train Epoch: 8 [17920/60000 (30%)]	Loss: 0.105314
Train Epoch: 8 [18560/60000 (31%)]	Loss: 0.207962
Train Epoch: 8 [19200/60000 (32%)]	Loss: 0.218575
Train Epoch: 8 [19840/60000 (33%)]	Loss: 0.168233
Train Epoch: 8 [20480/60000 (34%)]	Loss: 0.250853
Train Epoch: 8 [21120/60000 (35%)]	Loss: 0.220345
Train Epoch: 8 [21760/60000 (36%)]	Loss: 0.134018
Train Epoch: 8 [22400/60000 (37%)]	Loss: 0.185537
Train Epoch: 8 [23040/60000 (38%)]	Loss: 0.452978
Train Epoch: 8 [23680/60000 (39%)]	Loss: 0.139703
Train Epoch: 8 [24320/60000 (41%)]	Loss: 0.193488
Train Epoch: 8 [24960/60000 (42%)]	Loss: 0.377401


Train Epoch: 9 [11520/60000 (19%)]	Loss: 0.398446
Train Epoch: 9 [12160/60000 (20%)]	Loss: 0.195943
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.156539
Train Epoch: 9 [13440/60000 (22%)]	Loss: 0.209769
Train Epoch: 9 [14080/60000 (23%)]	Loss: 0.235748
Train Epoch: 9 [14720/60000 (25%)]	Loss: 0.177647
Train Epoch: 9 [15360/60000 (26%)]	Loss: 0.067535
Train Epoch: 9 [16000/60000 (27%)]	Loss: 0.138777
Train Epoch: 9 [16640/60000 (28%)]	Loss: 0.323216
Train Epoch: 9 [17280/60000 (29%)]	Loss: 0.237414
Train Epoch: 9 [17920/60000 (30%)]	Loss: 0.372122
Train Epoch: 9 [18560/60000 (31%)]	Loss: 0.202159
Train Epoch: 9 [19200/60000 (32%)]	Loss: 0.150903
Train Epoch: 9 [19840/60000 (33%)]	Loss: 0.410203
Train Epoch: 9 [20480/60000 (34%)]	Loss: 0.139419
Train Epoch: 9 [21120/60000 (35%)]	Loss: 0.153360
Train Epoch: 9 [21760/60000 (36%)]	Loss: 0.134512
Train Epoch: 9 [22400/60000 (37%)]	Loss: 0.306212
Train Epoch: 9 [23040/60000 (38%)]	Loss: 0.102832
Train Epoch: 9 [23680/60000 (39%)]	Loss: 0.180860


Train Epoch: 10 [12160/60000 (20%)]	Loss: 0.252697
Train Epoch: 10 [12800/60000 (21%)]	Loss: 0.078243
Train Epoch: 10 [13440/60000 (22%)]	Loss: 0.190291
Train Epoch: 10 [14080/60000 (23%)]	Loss: 0.092348
Train Epoch: 10 [14720/60000 (25%)]	Loss: 0.080954
Train Epoch: 10 [15360/60000 (26%)]	Loss: 0.330040
Train Epoch: 10 [16000/60000 (27%)]	Loss: 0.081626
Train Epoch: 10 [16640/60000 (28%)]	Loss: 0.190516
Train Epoch: 10 [17280/60000 (29%)]	Loss: 0.420826
Train Epoch: 10 [17920/60000 (30%)]	Loss: 0.248858
Train Epoch: 10 [18560/60000 (31%)]	Loss: 0.187518
Train Epoch: 10 [19200/60000 (32%)]	Loss: 0.228963
Train Epoch: 10 [19840/60000 (33%)]	Loss: 0.177074
Train Epoch: 10 [20480/60000 (34%)]	Loss: 0.126764
Train Epoch: 10 [21120/60000 (35%)]	Loss: 0.280106
Train Epoch: 10 [21760/60000 (36%)]	Loss: 0.071462
Train Epoch: 10 [22400/60000 (37%)]	Loss: 0.180942
Train Epoch: 10 [23040/60000 (38%)]	Loss: 0.126141
Train Epoch: 10 [23680/60000 (39%)]	Loss: 0.173247
Train Epoch: 10 [24320/60000 (4

### MNIST 테스트 (balancing train dataset)

In [0]:
def make_weights_for_balanced_classes(images, nclasses):
    count = [0] * nclasses
    for item in images:
        count[item[1]] += 1
    weight_per_class = [0.] * nclasses
    N = float(sum(count))
    for i in range(nclasses):
        weight_per_class[i] = N / float(count[i])
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[val[1]]
    return weight

# For unbalanced dataset we create a weighted sampler
weights = make_weights_for_balanced_classes(train_loader.dataset, 10)
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_loader = torch.utils.data.DataLoader(train_loader.dataset, batch_size=args.batch_size, shuffle = False,
                                                             sampler = sampler, **kwargs)

In [60]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.388022
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.322550
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.270663
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.243266
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.263846
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.218223
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.204652
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.186174
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.121039
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.131055
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.930972
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.796160
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.722322
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.690496
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.565326
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.463641
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.384629
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.366554
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.412691
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.081612
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.375118
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.373896
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.358470
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.542999
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.499460
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.430954
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.535538
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.346938
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.502906
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.599314
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.403539
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.311402
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.323650
Train Epoch: 2 [13440/60000 (22%)]	Loss: 0.482653
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.290528
Train Epoch: 2 [14720/60000 (25%)]	Loss: 0.366767
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.283897
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.359356
Train Epoch: 2 [16640/60000 (28%)]	Loss: 0.697285
Train Epoch: 2 [17280/60000 (29%)]	Loss: 0.292204
Train Epo

Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.222168
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.499232
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.294297
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.403682
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.402229
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.294917
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.330347
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.299236
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.356164
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.197017
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.461432
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.337282
Train Epoch: 3 [13440/60000 (22%)]	Loss: 0.186738
Train Epoch: 3 [14080/60000 (23%)]	Loss: 0.295718
Train Epoch: 3 [14720/60000 (25%)]	Loss: 0.406216
Train Epoch: 3 [15360/60000 (26%)]	Loss: 0.352355
Train Epoch: 3 [16000/60000 (27%)]	Loss: 0.255707
Train Epoch: 3 [16640/60000 (28%)]	Loss: 0.278529
Train Epoch: 3 [17280/60000 (29%)]	Loss: 0.295180
Train Epoch: 3 [17920/60000 (30%)]	Loss: 0.318722
Train E

Train Epoch: 4 [5760/60000 (10%)]	Loss: 0.220318
Train Epoch: 4 [6400/60000 (11%)]	Loss: 0.324852
Train Epoch: 4 [7040/60000 (12%)]	Loss: 0.560815
Train Epoch: 4 [7680/60000 (13%)]	Loss: 0.280115
Train Epoch: 4 [8320/60000 (14%)]	Loss: 0.247734
Train Epoch: 4 [8960/60000 (15%)]	Loss: 0.500088
Train Epoch: 4 [9600/60000 (16%)]	Loss: 0.330269
Train Epoch: 4 [10240/60000 (17%)]	Loss: 0.224629
Train Epoch: 4 [10880/60000 (18%)]	Loss: 0.236268
Train Epoch: 4 [11520/60000 (19%)]	Loss: 0.286820
Train Epoch: 4 [12160/60000 (20%)]	Loss: 0.141642
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.367445
Train Epoch: 4 [13440/60000 (22%)]	Loss: 0.287423
Train Epoch: 4 [14080/60000 (23%)]	Loss: 0.206744
Train Epoch: 4 [14720/60000 (25%)]	Loss: 0.266050
Train Epoch: 4 [15360/60000 (26%)]	Loss: 0.293356
Train Epoch: 4 [16000/60000 (27%)]	Loss: 0.188409
Train Epoch: 4 [16640/60000 (28%)]	Loss: 0.281854
Train Epoch: 4 [17280/60000 (29%)]	Loss: 0.241233
Train Epoch: 4 [17920/60000 (30%)]	Loss: 0.345584
Train E

Train Epoch: 5 [10880/60000 (18%)]	Loss: 0.250634
Train Epoch: 5 [11520/60000 (19%)]	Loss: 0.490823
Train Epoch: 5 [12160/60000 (20%)]	Loss: 0.173697
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.151127
Train Epoch: 5 [13440/60000 (22%)]	Loss: 0.092239
Train Epoch: 5 [14080/60000 (23%)]	Loss: 0.218804
Train Epoch: 5 [14720/60000 (25%)]	Loss: 0.188397
Train Epoch: 5 [15360/60000 (26%)]	Loss: 0.193022
Train Epoch: 5 [16000/60000 (27%)]	Loss: 0.142791
Train Epoch: 5 [16640/60000 (28%)]	Loss: 0.198917
Train Epoch: 5 [17280/60000 (29%)]	Loss: 0.292849
Train Epoch: 5 [17920/60000 (30%)]	Loss: 0.293777
Train Epoch: 5 [18560/60000 (31%)]	Loss: 0.154192
Train Epoch: 5 [19200/60000 (32%)]	Loss: 0.183185
Train Epoch: 5 [19840/60000 (33%)]	Loss: 0.148647
Train Epoch: 5 [20480/60000 (34%)]	Loss: 0.214932
Train Epoch: 5 [21120/60000 (35%)]	Loss: 0.162598
Train Epoch: 5 [21760/60000 (36%)]	Loss: 0.269232
Train Epoch: 5 [22400/60000 (37%)]	Loss: 0.179063
Train Epoch: 5 [23040/60000 (38%)]	Loss: 0.240292


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.221621
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.140927
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.267833
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.244650
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.199957
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.149615
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.159526
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.273155
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.334807
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.214314
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.251980
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.259537
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.267220
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.071128
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.193310
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.126830
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.372391
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.140784
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.160494
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.323606


Train Epoch: 7 [9600/60000 (16%)]	Loss: 0.316561
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.147710
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.229414
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.096121
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.246527
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.277814
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.324015
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.125243
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.307518
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.233250
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.159233
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.199545
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.203297
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.114171
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.143871
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.249078
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.350922
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.188498
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.240010
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.263729
T

Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.086577
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.217617
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.043505
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.146771
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.170627
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.228412
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.207028
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.367381
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.232891
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.069340
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.100920
Train Epoch: 8 [12160/60000 (20%)]	Loss: 0.280416
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.232370
Train Epoch: 8 [13440/60000 (22%)]	Loss: 0.177867
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.117884
Train Epoch: 8 [14720/60000 (25%)]	Loss: 0.214655
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.113880
Train Epoch: 8 [16000/60000 (27%)]	Loss: 0.130230
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.236809
Train Epoch: 8 [17280/60000 (29%)]	Loss: 0.097824
Train Epo

Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.167076
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.103090
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.266793
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.215962
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.091075
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.093729
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.207097
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.094140
Train Epoch: 9 [9600/60000 (16%)]	Loss: 0.128751
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.281951
Train Epoch: 9 [10880/60000 (18%)]	Loss: 0.037557
Train Epoch: 9 [11520/60000 (19%)]	Loss: 0.211944
Train Epoch: 9 [12160/60000 (20%)]	Loss: 0.172245
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.258246
Train Epoch: 9 [13440/60000 (22%)]	Loss: 0.140205
Train Epoch: 9 [14080/60000 (23%)]	Loss: 0.182274
Train Epoch: 9 [14720/60000 (25%)]	Loss: 0.321574
Train Epoch: 9 [15360/60000 (26%)]	Loss: 0.122765
Train Epoch: 9 [16000/60000 (27%)]	Loss: 0.157090
Train Epoch: 9 [16640/60000 (28%)]	Loss: 0.272698
Train Epoch

Train Epoch: 10 [8320/60000 (14%)]	Loss: 0.324444
Train Epoch: 10 [8960/60000 (15%)]	Loss: 0.143952
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.046148
Train Epoch: 10 [10240/60000 (17%)]	Loss: 0.162849
Train Epoch: 10 [10880/60000 (18%)]	Loss: 0.091458
Train Epoch: 10 [11520/60000 (19%)]	Loss: 0.417716
Train Epoch: 10 [12160/60000 (20%)]	Loss: 0.247339
Train Epoch: 10 [12800/60000 (21%)]	Loss: 0.160072
Train Epoch: 10 [13440/60000 (22%)]	Loss: 0.122420
Train Epoch: 10 [14080/60000 (23%)]	Loss: 0.085987
Train Epoch: 10 [14720/60000 (25%)]	Loss: 0.190538
Train Epoch: 10 [15360/60000 (26%)]	Loss: 0.079759
Train Epoch: 10 [16000/60000 (27%)]	Loss: 0.089574
Train Epoch: 10 [16640/60000 (28%)]	Loss: 0.112082
Train Epoch: 10 [17280/60000 (29%)]	Loss: 0.168163
Train Epoch: 10 [17920/60000 (30%)]	Loss: 0.218057
Train Epoch: 10 [18560/60000 (31%)]	Loss: 0.160999
Train Epoch: 10 [19200/60000 (32%)]	Loss: 0.193268
Train Epoch: 10 [19840/60000 (33%)]	Loss: 0.200962
Train Epoch: 10 [20480/60000 (34%)

### MNIST 테스트 (imbalancing train dataset) 샘플링 확률 : class별 [0.9, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

In [0]:
def make_weights_for_balanced_classes(images, nclasses):
    weight_per_class = [0.9, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[val[1]]
    return weight

# For unbalanced dataset we create a weighted sampler
weights = make_weights_for_balanced_classes(train_loader.dataset, 10)
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_loader = torch.utils.data.DataLoader(train_loader.dataset, batch_size=args.batch_size, shuffle = False,
                                                             sampler = sampler, **kwargs)

In [100]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.387138
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.468628
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.387452
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.661923
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.750759
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.480220
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.404496
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.504542
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.589429
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.518735
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.321070
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.525253
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.213040
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.345564
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.505251
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.423952
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.078382
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.230174
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.201633
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.289445
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.161310
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.222728
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.053702
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.059440
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.086131
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.171599
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.196152
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.173222
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.066826
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.136361
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.188878
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.232339
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.088650
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.153361
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.061734
Train Epoch: 2 [13440/60000 (22%)]	Loss: 0.159013
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.273869
Train Epoch: 2 [14720/60000 (25%)]	Loss: 0.149017
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.198010
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.196013
Train Epoch: 

Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.134847
Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.119856
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.032886
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.278450
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.065486
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.056511
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.088038
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.280396
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.142241
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.059984
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.182539
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.246602
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.046882
Train Epoch: 3 [13440/60000 (22%)]	Loss: 0.056674
Train Epoch: 3 [14080/60000 (23%)]	Loss: 0.215140
Train Epoch: 3 [14720/60000 (25%)]	Loss: 0.116042
Train Epoch: 3 [15360/60000 (26%)]	Loss: 0.105046
Train Epoch: 3 [16000/60000 (27%)]	Loss: 0.249953
Train Epoch: 3 [16640/60000 (28%)]	Loss: 0.063738
Train Epoch: 3 [17280/60000 (29%)]	Loss: 0.007687
Train Epo

Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.051485
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.134363
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.099848
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.166859
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.048358
Train Epoch: 3 [56960/60000 (95%)]	Loss: 0.053916
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.040333
Train Epoch: 3 [58240/60000 (97%)]	Loss: 0.066978
Train Epoch: 3 [58880/60000 (98%)]	Loss: 0.161330
Train Epoch: 3 [59520/60000 (99%)]	Loss: 0.043297

Test set: Average loss: 0.5832, Accuracy: 8476/10000 (85%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.110769
Train Epoch: 4 [640/60000 (1%)]	Loss: 0.102456
Train Epoch: 4 [1280/60000 (2%)]	Loss: 0.074638
Train Epoch: 4 [1920/60000 (3%)]	Loss: 0.094733
Train Epoch: 4 [2560/60000 (4%)]	Loss: 0.061025
Train Epoch: 4 [3200/60000 (5%)]	Loss: 0.112034
Train Epoch: 4 [3840/60000 (6%)]	Loss: 0.191285
Train Epoch: 4 [4480/60000 (7%)]	Loss: 0.167042
Train Epoch: 4 [5120/60000 (9%)]	Loss: 0.184206
Train Epoch


Test set: Average loss: 0.4492, Accuracy: 8792/10000 (88%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.025635
Train Epoch: 5 [640/60000 (1%)]	Loss: 0.223442
Train Epoch: 5 [1280/60000 (2%)]	Loss: 0.010185
Train Epoch: 5 [1920/60000 (3%)]	Loss: 0.069028
Train Epoch: 5 [2560/60000 (4%)]	Loss: 0.187343
Train Epoch: 5 [3200/60000 (5%)]	Loss: 0.160187
Train Epoch: 5 [3840/60000 (6%)]	Loss: 0.017902
Train Epoch: 5 [4480/60000 (7%)]	Loss: 0.046595
Train Epoch: 5 [5120/60000 (9%)]	Loss: 0.031944
Train Epoch: 5 [5760/60000 (10%)]	Loss: 0.366310
Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.044605
Train Epoch: 5 [7040/60000 (12%)]	Loss: 0.087137
Train Epoch: 5 [7680/60000 (13%)]	Loss: 0.070450
Train Epoch: 5 [8320/60000 (14%)]	Loss: 0.102085
Train Epoch: 5 [8960/60000 (15%)]	Loss: 0.073278
Train Epoch: 5 [9600/60000 (16%)]	Loss: 0.074062
Train Epoch: 5 [10240/60000 (17%)]	Loss: 0.070407
Train Epoch: 5 [10880/60000 (18%)]	Loss: 0.144789
Train Epoch: 5 [11520/60000 (19%)]	Loss: 0.101253
Train Epoch: 5 [12

Train Epoch: 5 [47360/60000 (79%)]	Loss: 0.019150
Train Epoch: 5 [48000/60000 (80%)]	Loss: 0.086425
Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.084802
Train Epoch: 5 [49280/60000 (82%)]	Loss: 0.118591
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.052752
Train Epoch: 5 [50560/60000 (84%)]	Loss: 0.075522
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.092885
Train Epoch: 5 [51840/60000 (86%)]	Loss: 0.041292
Train Epoch: 5 [52480/60000 (87%)]	Loss: 0.117125
Train Epoch: 5 [53120/60000 (88%)]	Loss: 0.052316
Train Epoch: 5 [53760/60000 (90%)]	Loss: 0.095850
Train Epoch: 5 [54400/60000 (91%)]	Loss: 0.086633
Train Epoch: 5 [55040/60000 (92%)]	Loss: 0.084319
Train Epoch: 5 [55680/60000 (93%)]	Loss: 0.024037
Train Epoch: 5 [56320/60000 (94%)]	Loss: 0.347200
Train Epoch: 5 [56960/60000 (95%)]	Loss: 0.025510
Train Epoch: 5 [57600/60000 (96%)]	Loss: 0.160859
Train Epoch: 5 [58240/60000 (97%)]	Loss: 0.071675
Train Epoch: 5 [58880/60000 (98%)]	Loss: 0.089885
Train Epoch: 5 [59520/60000 (99%)]	Loss: 0.088157


Train Epoch: 6 [53120/60000 (88%)]	Loss: 0.011078
Train Epoch: 6 [53760/60000 (90%)]	Loss: 0.042509
Train Epoch: 6 [54400/60000 (91%)]	Loss: 0.056562
Train Epoch: 6 [55040/60000 (92%)]	Loss: 0.088498
Train Epoch: 6 [55680/60000 (93%)]	Loss: 0.031377
Train Epoch: 6 [56320/60000 (94%)]	Loss: 0.150518
Train Epoch: 6 [56960/60000 (95%)]	Loss: 0.109855
Train Epoch: 6 [57600/60000 (96%)]	Loss: 0.033439
Train Epoch: 6 [58240/60000 (97%)]	Loss: 0.151980
Train Epoch: 6 [58880/60000 (98%)]	Loss: 0.255537
Train Epoch: 6 [59520/60000 (99%)]	Loss: 0.106183

Test set: Average loss: 0.3461, Accuracy: 9092/10000 (91%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.160418
Train Epoch: 7 [640/60000 (1%)]	Loss: 0.026107
Train Epoch: 7 [1280/60000 (2%)]	Loss: 0.081960
Train Epoch: 7 [1920/60000 (3%)]	Loss: 0.091835
Train Epoch: 7 [2560/60000 (4%)]	Loss: 0.095147
Train Epoch: 7 [3200/60000 (5%)]	Loss: 0.063406
Train Epoch: 7 [3840/60000 (6%)]	Loss: 0.155956
Train Epoch: 7 [4480/60000 (7%)]	Loss: 0.046177
Train Epo

Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.053980
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.054753

Test set: Average loss: 0.3003, Accuracy: 9248/10000 (92%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.051608
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.033328
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.031141
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.105774
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.163415
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.075394
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.051697
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.067993
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.058373
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.112347
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.145357
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.015206
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.053815
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.101296
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.015512
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.019358
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.083709
Train Epoch: 8 [10

Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.185217
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.182952
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.118131
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.139235
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.022636
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.084537
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.047955
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.043876
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.039399
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.080745
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.057469
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.170450
Train Epoch: 9 [8960/60000 (15%)]	Loss: 0.060086
Train Epoch: 9 [9600/60000 (16%)]	Loss: 0.064542
Train Epoch: 9 [10240/60000 (17%)]	Loss: 0.055901
Train Epoch: 9 [10880/60000 (18%)]	Loss: 0.041264
Train Epoch: 9 [11520/60000 (19%)]	Loss: 0.021008
Train Epoch: 9 [12160/60000 (20%)]	Loss: 0.086734
Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.100308
Train Epoch: 9 [13440/60000 (22%)]	Loss: 0.030313
Train Epoch: 9 [14080

Train Epoch: 10 [3840/60000 (6%)]	Loss: 0.221666
Train Epoch: 10 [4480/60000 (7%)]	Loss: 0.251034
Train Epoch: 10 [5120/60000 (9%)]	Loss: 0.074214
Train Epoch: 10 [5760/60000 (10%)]	Loss: 0.109434
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.003190
Train Epoch: 10 [7040/60000 (12%)]	Loss: 0.028057
Train Epoch: 10 [7680/60000 (13%)]	Loss: 0.018047
Train Epoch: 10 [8320/60000 (14%)]	Loss: 0.054378
Train Epoch: 10 [8960/60000 (15%)]	Loss: 0.000881
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.055376
Train Epoch: 10 [10240/60000 (17%)]	Loss: 0.056424
Train Epoch: 10 [10880/60000 (18%)]	Loss: 0.062063
Train Epoch: 10 [11520/60000 (19%)]	Loss: 0.004802
Train Epoch: 10 [12160/60000 (20%)]	Loss: 0.061137
Train Epoch: 10 [12800/60000 (21%)]	Loss: 0.055804
Train Epoch: 10 [13440/60000 (22%)]	Loss: 0.031419
Train Epoch: 10 [14080/60000 (23%)]	Loss: 0.021096
Train Epoch: 10 [14720/60000 (25%)]	Loss: 0.061231
Train Epoch: 10 [15360/60000 (26%)]	Loss: 0.009738
Train Epoch: 10 [16000/60000 (27%)]	Loss: 0.

### MNIST 테스트 (imbalancing train dataset) 샘플링 확률 : class별 [0.99, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]

In [0]:
def make_weights_for_balanced_classes(images, nclasses):
    weight_per_class = [0.99, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[val[1]]
    return weight

# For unbalanced dataset we create a weighted sampler
weights = make_weights_for_balanced_classes(train_loader.dataset, 10)
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_loader = torch.utils.data.DataLoader(train_loader.dataset, batch_size=args.batch_size, shuffle = False,
                                                             sampler = sampler, **kwargs)

In [103]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.525626
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.377094
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.078198
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.074147
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.112585
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.121647
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.051652
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.067130
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.025575
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.052903
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.236640
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.009028
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.110571
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.192951
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.013879
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.378845
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.060969
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.023663
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.021902
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.011803
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.003305
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.082519
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.044341
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.013394
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.002795
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.095135
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.182697
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.117764
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.014838
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.007441
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.002109
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.001822
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.066754
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.129331
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.015022
Train Epoch: 2 [13440/60000 (22%)]	Loss: 0.029639
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.084607
Train Epoch: 2 [14720/60000 (25%)]	Loss: 0.001867
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.018393
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.003677
Train Epoch: 

Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.007481
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.060032
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.030723
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.009108
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.012020
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.062780
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.000725
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.036550
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.016101
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.031061
Train Epoch: 2 [58880/60000 (98%)]	Loss: 0.045253
Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.051268

Test set: Average loss: 2.8094, Accuracy: 2326/10000 (23%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.004854
Train Epoch: 3 [640/60000 (1%)]	Loss: 0.004726
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.108545
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.012789
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.004114
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.004104
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.005540
Train E

Train Epoch: 3 [40320/60000 (67%)]	Loss: 0.000151
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.120928
Train Epoch: 3 [41600/60000 (69%)]	Loss: 0.082829
Train Epoch: 3 [42240/60000 (70%)]	Loss: 0.001299
Train Epoch: 3 [42880/60000 (71%)]	Loss: 0.025687
Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.038195
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.119466
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.077486
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.041412
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.220609
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.023915
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.047814
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.037480
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.004708
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.004163
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.044770
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.072401
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.097853
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.000639
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.004101


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.017303
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.017132
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.005024
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.050683
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.054391
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.000738
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.100524
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.002908
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.003555
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.016091
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.000174
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.052405
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.000947
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.000797
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.045706
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.041704
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.000456
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.001495
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.066802
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.151130


...

Train Epoch: 7 [5120/60000 (9%)]	Loss: 0.001285
Train Epoch: 7 [5760/60000 (10%)]	Loss: 0.001638
Train Epoch: 7 [6400/60000 (11%)]	Loss: 0.002157
Train Epoch: 7 [7040/60000 (12%)]	Loss: 0.000628
Train Epoch: 7 [7680/60000 (13%)]	Loss: 0.003368
Train Epoch: 7 [8320/60000 (14%)]	Loss: 0.001442
Train Epoch: 7 [8960/60000 (15%)]	Loss: 0.001441
Train Epoch: 7 [9600/60000 (16%)]	Loss: 0.000225
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.000336
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.034091
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.002533
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.076019
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.005631
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.000060
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.002163
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.001380
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.056770
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.000377
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.025449
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.007834
Train Epo

Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.000507
Train Epoch: 7 [54400/60000 (91%)]	Loss: 0.024277
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.104804
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.000542
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.047485
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.000435
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.041755
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.032097
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.000665
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.000203

Test set: Average loss: 1.8095, Accuracy: 5313/10000 (53%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.000234
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.002383
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.044302
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.086912
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.001925
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.048598
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.054357
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.008820
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.001655
Train Epoch

Train Epoch: 8 [41600/60000 (69%)]	Loss: 0.048421
Train Epoch: 8 [42240/60000 (70%)]	Loss: 0.040130
Train Epoch: 8 [42880/60000 (71%)]	Loss: 0.082254
Train Epoch: 8 [43520/60000 (72%)]	Loss: 0.002976
Train Epoch: 8 [44160/60000 (74%)]	Loss: 0.000920
Train Epoch: 8 [44800/60000 (75%)]	Loss: 0.003395
Train Epoch: 8 [45440/60000 (76%)]	Loss: 0.030790
Train Epoch: 8 [46080/60000 (77%)]	Loss: 0.115669
Train Epoch: 8 [46720/60000 (78%)]	Loss: 0.049002
Train Epoch: 8 [47360/60000 (79%)]	Loss: 0.007967
Train Epoch: 8 [48000/60000 (80%)]	Loss: 0.076233
Train Epoch: 8 [48640/60000 (81%)]	Loss: 0.034849
Train Epoch: 8 [49280/60000 (82%)]	Loss: 0.064440
Train Epoch: 8 [49920/60000 (83%)]	Loss: 0.002629
Train Epoch: 8 [50560/60000 (84%)]	Loss: 0.000503
Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.061532
Train Epoch: 8 [51840/60000 (86%)]	Loss: 0.001279
Train Epoch: 8 [52480/60000 (87%)]	Loss: 0.000009
Train Epoch: 8 [53120/60000 (88%)]	Loss: 0.062267
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.018162


Train Epoch: 9 [28800/60000 (48%)]	Loss: 0.001497
Train Epoch: 9 [29440/60000 (49%)]	Loss: 0.025520
Train Epoch: 9 [30080/60000 (50%)]	Loss: 0.000927
Train Epoch: 9 [30720/60000 (51%)]	Loss: 0.053818
Train Epoch: 9 [31360/60000 (52%)]	Loss: 0.001078
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.002601
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.001246
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.004294
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.000569
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.000012
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.035937
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.064270
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.055089
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.040472
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.000901
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.001508
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.067634
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.001499
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.005908
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.123188


Train Epoch: 10 [17280/60000 (29%)]	Loss: 0.005457
Train Epoch: 10 [17920/60000 (30%)]	Loss: 0.064958
Train Epoch: 10 [18560/60000 (31%)]	Loss: 0.001266
Train Epoch: 10 [19200/60000 (32%)]	Loss: 0.000650
Train Epoch: 10 [19840/60000 (33%)]	Loss: 0.042898
Train Epoch: 10 [20480/60000 (34%)]	Loss: 0.016849
Train Epoch: 10 [21120/60000 (35%)]	Loss: 0.113662
Train Epoch: 10 [21760/60000 (36%)]	Loss: 0.037160
Train Epoch: 10 [22400/60000 (37%)]	Loss: 0.002340
Train Epoch: 10 [23040/60000 (38%)]	Loss: 0.095803
Train Epoch: 10 [23680/60000 (39%)]	Loss: 0.015753
Train Epoch: 10 [24320/60000 (41%)]	Loss: 0.000200
Train Epoch: 10 [24960/60000 (42%)]	Loss: 0.008437
Train Epoch: 10 [25600/60000 (43%)]	Loss: 0.000360
Train Epoch: 10 [26240/60000 (44%)]	Loss: 0.000282
Train Epoch: 10 [26880/60000 (45%)]	Loss: 0.011424
Train Epoch: 10 [27520/60000 (46%)]	Loss: 0.046833
Train Epoch: 10 [28160/60000 (47%)]	Loss: 0.011793
Train Epoch: 10 [28800/60000 (48%)]	Loss: 0.000372
Train Epoch: 10 [29440/60000 (4

### MNIST 테스트 (imbalancing train dataset) 샘플링 확률 : class별 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [0]:
def make_weights_for_balanced_classes(images, nclasses):
    weight_per_class = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[val[1]]
    return weight

# For unbalanced dataset we create a weighted sampler
weights = make_weights_for_balanced_classes(train_loader.dataset, 10)
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_loader = torch.utils.data.DataLoader(train_loader.dataset, batch_size=args.batch_size, shuffle = False,
                                                             sampler = sampler, **kwargs)

In [89]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.000003
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.000066
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.000002
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.000001
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.000000
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.000003
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.000172
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.000000
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.000031
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.000000
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.000000
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.000002
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.000001
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.003817
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.000000
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.000558
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.000000
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.000007
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.000003
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.000000
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.000005
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.000000
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.000003
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.000007
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.000000
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.002492
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.000045
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.000000
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.000001
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.000003
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.000048
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.000194
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.000135
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.000003
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.000000
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.000001
Train Epoch: 2 [13440/60000 (22%)]	Loss: 0.000218
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.000000
Train Epoch: 2 [14720/60000 (25%)]	Loss: 0.000001
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.000002
Train Epoch: 2 

Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.000001
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.000034
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.000000
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.000043
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.000000
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.000000
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.000000
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.000000
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.000000
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.000000
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.000000
Train Epoch: 2 [58880/60000 (98%)]	Loss: 0.000006
Train Epoch: 2 [59520/60000 (99%)]	Loss: 0.000002

Test set: Average loss: 42.3182, Accuracy: 980/10000 (10%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.000003
Train Epoch: 3 [640/60000 (1%)]	Loss: 0.000089
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.000000
Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.000000
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.000000
Train Epoch: 3 [3200/60000 (5%)]	Loss: 0.000000
Train

Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.000000
Train Epoch: 3 [40320/60000 (67%)]	Loss: 0.000342
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.000010
Train Epoch: 3 [41600/60000 (69%)]	Loss: 0.000000
Train Epoch: 3 [42240/60000 (70%)]	Loss: 0.000000
Train Epoch: 3 [42880/60000 (71%)]	Loss: 0.000000
Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.000000
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.000000
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.000000
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.000000
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.000000
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.000000
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.000027
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.000000
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.000003
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.000139
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.000000
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.000000
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.000000
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.000286


Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.000000
Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.000000
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.000000
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.000001
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.000000
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.000000
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.000026
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.000000
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.000000
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.000000
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.000006
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.000000
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.000000
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.000001
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.000084
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.000000
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.000000
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.000000
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.000004
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.000000


...


Test set: Average loss: 46.2153, Accuracy: 980/10000 (10%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.000061
Train Epoch: 7 [640/60000 (1%)]	Loss: 0.000000
Train Epoch: 7 [1280/60000 (2%)]	Loss: 0.000000
Train Epoch: 7 [1920/60000 (3%)]	Loss: 0.000000
Train Epoch: 7 [2560/60000 (4%)]	Loss: 0.000002
Train Epoch: 7 [3200/60000 (5%)]	Loss: 0.000000
Train Epoch: 7 [3840/60000 (6%)]	Loss: 0.000000
Train Epoch: 7 [4480/60000 (7%)]	Loss: 0.000000
Train Epoch: 7 [5120/60000 (9%)]	Loss: 0.000000
Train Epoch: 7 [5760/60000 (10%)]	Loss: 0.000000
Train Epoch: 7 [6400/60000 (11%)]	Loss: 0.000003
Train Epoch: 7 [7040/60000 (12%)]	Loss: 0.000000
Train Epoch: 7 [7680/60000 (13%)]	Loss: 0.000001
Train Epoch: 7 [8320/60000 (14%)]	Loss: 0.000021
Train Epoch: 7 [8960/60000 (15%)]	Loss: 0.000018
Train Epoch: 7 [9600/60000 (16%)]	Loss: 0.000000
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.000000
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.002426
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.000000
Train Epoch: 7 [12

Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.000000
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.000051

Test set: Average loss: 46.9251, Accuracy: 980/10000 (10%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.000001
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.000000
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.000000
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.000000
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.000000
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.000000
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.000000
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.000000
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.000000
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.000000
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.000000
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.000000
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.000005
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.000004
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.000505
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.000000
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.000000
Train Epoch: 8 [10

Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.000036
Train Epoch: 8 [51840/60000 (86%)]	Loss: 0.000000
Train Epoch: 8 [52480/60000 (87%)]	Loss: 0.000000
Train Epoch: 8 [53120/60000 (88%)]	Loss: 0.000000
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.000001
Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.000000
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.000000
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.000002
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.000000
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.000009
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.000000
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.000001
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.000000
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.000000

Test set: Average loss: 47.5256, Accuracy: 980/10000 (10%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.000000
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.000001
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.000000
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.000001
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.000001
Tra

Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.000000
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.000000
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.000000
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.000000
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.000001
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.000000
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.000000
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.000000
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.000000
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.000000
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.000000
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.000011
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.000000
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.000000
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.000000
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.000006
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.000032
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.000000
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.000005
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.000000


Train Epoch: 10 [26880/60000 (45%)]	Loss: 0.000000
Train Epoch: 10 [27520/60000 (46%)]	Loss: 0.000000
Train Epoch: 10 [28160/60000 (47%)]	Loss: 0.000000
Train Epoch: 10 [28800/60000 (48%)]	Loss: 0.000000
Train Epoch: 10 [29440/60000 (49%)]	Loss: 0.000000
Train Epoch: 10 [30080/60000 (50%)]	Loss: 0.000000
Train Epoch: 10 [30720/60000 (51%)]	Loss: 0.000000
Train Epoch: 10 [31360/60000 (52%)]	Loss: 0.000000
Train Epoch: 10 [32000/60000 (53%)]	Loss: 0.000000
Train Epoch: 10 [32640/60000 (54%)]	Loss: 0.000000
Train Epoch: 10 [33280/60000 (55%)]	Loss: 0.000000
Train Epoch: 10 [33920/60000 (57%)]	Loss: 0.000000
Train Epoch: 10 [34560/60000 (58%)]	Loss: 0.000000
Train Epoch: 10 [35200/60000 (59%)]	Loss: 0.000000
Train Epoch: 10 [35840/60000 (60%)]	Loss: 0.000000
Train Epoch: 10 [36480/60000 (61%)]	Loss: 0.000000
Train Epoch: 10 [37120/60000 (62%)]	Loss: 0.000007
Train Epoch: 10 [37760/60000 (63%)]	Loss: 0.000000
Train Epoch: 10 [38400/60000 (64%)]	Loss: 0.000000
Train Epoch: 10 [39040/60000 (6

### MNIST 테스트 (imbalancing train dataset) 샘플링 확률 : class별 (Kaggle Top10 Label과 동일) 
[0.24707459, 0.2461469 , 0.1149304 , 0.09066322, 0.0651395, 0.05471404, 0.04666915, 0.04526535, 0.0452359 , 0.04416096]

In [119]:
a = [50337,50148,23415,18471,13271,11147,9508,9222,9216,8997]
na = np.array(a)
na/na.sum()

array([0.24707459, 0.2461469 , 0.1149304 , 0.09066322, 0.0651395 ,
       0.05471404, 0.04666915, 0.04526535, 0.0452359 , 0.04416096])

In [0]:
def make_weights_for_balanced_classes(images, nclasses):
    weight_per_class = [0.24707459, 0.2461469 , 0.1149304 , 0.09066322, 0.0651395, 0.05471404, 0.04666915, 0.04526535, 0.0452359 , 0.04416096]
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[val[1]]
    return weight

# For unbalanced dataset we create a weighted sampler
weights = make_weights_for_balanced_classes(train_loader.dataset, 10)
weights = torch.DoubleTensor(weights)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_loader = torch.utils.data.DataLoader(train_loader.dataset, batch_size=args.batch_size, shuffle = False,
                                                             sampler = sampler, **kwargs)

In [122]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.285530
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.203700
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.166321
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.059309
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.924441
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.573880
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.506950
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.600018
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.200832
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.489376
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.214119
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.097956
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.132540
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.016081
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.008698
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.090880
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.775446
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.358282
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.732452
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.104736
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.277334
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.360315
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.209900
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.433072
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.234495
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.314208
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.362186
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.320030
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.266110
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.143172
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.490422
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.415602
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.572371
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.288853
Train Epoch: 2 [12160/60000 (20%)]	Loss: 0.320513
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.329969
Train Epoch: 2 [13440/60000 (22%)]	Loss: 0.311488
Train Epoch: 2 [14080/60000 (23%)]	Loss: 0.134818
Train Epoch: 2 [14720/60000 (25%)]	Loss: 0.302103
Train Epoch: 2 [15360/60000 (26%)]	Loss: 0.292710
Train Epoch: 2 

Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.164289
Train Epoch: 3 [5760/60000 (10%)]	Loss: 0.374857
Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.219624
Train Epoch: 3 [7040/60000 (12%)]	Loss: 0.234254
Train Epoch: 3 [7680/60000 (13%)]	Loss: 0.219439
Train Epoch: 3 [8320/60000 (14%)]	Loss: 0.264798
Train Epoch: 3 [8960/60000 (15%)]	Loss: 0.128152
Train Epoch: 3 [9600/60000 (16%)]	Loss: 0.292716
Train Epoch: 3 [10240/60000 (17%)]	Loss: 0.283330
Train Epoch: 3 [10880/60000 (18%)]	Loss: 0.285260
Train Epoch: 3 [11520/60000 (19%)]	Loss: 0.282687
Train Epoch: 3 [12160/60000 (20%)]	Loss: 0.335369
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.219810
Train Epoch: 3 [13440/60000 (22%)]	Loss: 0.222659
Train Epoch: 3 [14080/60000 (23%)]	Loss: 0.338942
Train Epoch: 3 [14720/60000 (25%)]	Loss: 0.265283
Train Epoch: 3 [15360/60000 (26%)]	Loss: 0.226590
Train Epoch: 3 [16000/60000 (27%)]	Loss: 0.289736
Train Epoch: 3 [16640/60000 (28%)]	Loss: 0.137537
Train Epoch: 3 [17280/60000 (29%)]	Loss: 0.192467
Train Epo

Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.169400
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.668157
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.172512
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.599762
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.292089
Train Epoch: 3 [56960/60000 (95%)]	Loss: 0.479530
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.149415
Train Epoch: 3 [58240/60000 (97%)]	Loss: 0.274286
Train Epoch: 3 [58880/60000 (98%)]	Loss: 0.238472
Train Epoch: 3 [59520/60000 (99%)]	Loss: 0.212049

Test set: Average loss: 0.1413, Accuracy: 9587/10000 (96%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.413856
Train Epoch: 4 [640/60000 (1%)]	Loss: 0.245175
Train Epoch: 4 [1280/60000 (2%)]	Loss: 0.130729
Train Epoch: 4 [1920/60000 (3%)]	Loss: 0.142330
Train Epoch: 4 [2560/60000 (4%)]	Loss: 0.187724
Train Epoch: 4 [3200/60000 (5%)]	Loss: 0.350987
Train Epoch: 4 [3840/60000 (6%)]	Loss: 0.306733
Train Epoch: 4 [4480/60000 (7%)]	Loss: 0.201615
Train Epoch: 4 [5120/60000 (9%)]	Loss: 0.186631
Train Epoch

Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.189034
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.111126
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.155609
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.195396
Train Epoch: 4 [44160/60000 (74%)]	Loss: 0.101170
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.173303
Train Epoch: 4 [45440/60000 (76%)]	Loss: 0.080584
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.144509
Train Epoch: 4 [46720/60000 (78%)]	Loss: 0.110683
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.114574
Train Epoch: 4 [48000/60000 (80%)]	Loss: 0.196711
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.203425
Train Epoch: 4 [49280/60000 (82%)]	Loss: 0.118782
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.235309
Train Epoch: 4 [50560/60000 (84%)]	Loss: 0.221279
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.140169
Train Epoch: 4 [51840/60000 (86%)]	Loss: 0.074541
Train Epoch: 4 [52480/60000 (87%)]	Loss: 0.080125
Train Epoch: 4 [53120/60000 (88%)]	Loss: 0.299715
Train Epoch: 4 [53760/60000 (90%)]	Loss: 0.236117


...

Train Epoch: 7 [7680/60000 (13%)]	Loss: 0.221045
Train Epoch: 7 [8320/60000 (14%)]	Loss: 0.161176
Train Epoch: 7 [8960/60000 (15%)]	Loss: 0.066833
Train Epoch: 7 [9600/60000 (16%)]	Loss: 0.170725
Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.184026
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.071234
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.113596
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.142424
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.129989
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.237779
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.116782
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.141687
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.112804
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.163935
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.194724
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.114517
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.188141
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.157505
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.770821
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.101966
Trai

Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.264778
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.432022
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.193595
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.105273
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.166933
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.152239

Test set: Average loss: 0.0949, Accuracy: 9695/10000 (97%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.092822
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.096346
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.115784
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.163290
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.094959
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.173921
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.160605
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.180470
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.097087
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.176516
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.127850
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.089755
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.199921
Train Epoch: 8 

Train Epoch: 8 [43520/60000 (72%)]	Loss: 0.191193
Train Epoch: 8 [44160/60000 (74%)]	Loss: 0.056956
Train Epoch: 8 [44800/60000 (75%)]	Loss: 0.111396
Train Epoch: 8 [45440/60000 (76%)]	Loss: 0.238901
Train Epoch: 8 [46080/60000 (77%)]	Loss: 0.118953
Train Epoch: 8 [46720/60000 (78%)]	Loss: 0.197016
Train Epoch: 8 [47360/60000 (79%)]	Loss: 0.187814
Train Epoch: 8 [48000/60000 (80%)]	Loss: 0.033442
Train Epoch: 8 [48640/60000 (81%)]	Loss: 0.358698
Train Epoch: 8 [49280/60000 (82%)]	Loss: 0.085830
Train Epoch: 8 [49920/60000 (83%)]	Loss: 0.149129
Train Epoch: 8 [50560/60000 (84%)]	Loss: 0.113998
Train Epoch: 8 [51200/60000 (85%)]	Loss: 0.114529
Train Epoch: 8 [51840/60000 (86%)]	Loss: 0.207438
Train Epoch: 8 [52480/60000 (87%)]	Loss: 0.186352
Train Epoch: 8 [53120/60000 (88%)]	Loss: 0.323485
Train Epoch: 8 [53760/60000 (90%)]	Loss: 0.165195
Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.078039
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.126137
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.155922


Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.129473
Train Epoch: 9 [32640/60000 (54%)]	Loss: 0.181875
Train Epoch: 9 [33280/60000 (55%)]	Loss: 0.164591
Train Epoch: 9 [33920/60000 (57%)]	Loss: 0.119554
Train Epoch: 9 [34560/60000 (58%)]	Loss: 0.119060
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.204505
Train Epoch: 9 [35840/60000 (60%)]	Loss: 0.084254
Train Epoch: 9 [36480/60000 (61%)]	Loss: 0.116148
Train Epoch: 9 [37120/60000 (62%)]	Loss: 0.158536
Train Epoch: 9 [37760/60000 (63%)]	Loss: 0.068845
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.038486
Train Epoch: 9 [39040/60000 (65%)]	Loss: 0.325916
Train Epoch: 9 [39680/60000 (66%)]	Loss: 0.347449
Train Epoch: 9 [40320/60000 (67%)]	Loss: 0.280596
Train Epoch: 9 [40960/60000 (68%)]	Loss: 0.210642
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.143666
Train Epoch: 9 [42240/60000 (70%)]	Loss: 0.183140
Train Epoch: 9 [42880/60000 (71%)]	Loss: 0.133653
Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.234328
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.060032


Train Epoch: 10 [20480/60000 (34%)]	Loss: 0.138884
Train Epoch: 10 [21120/60000 (35%)]	Loss: 0.122684
Train Epoch: 10 [21760/60000 (36%)]	Loss: 0.041884
Train Epoch: 10 [22400/60000 (37%)]	Loss: 0.052837
Train Epoch: 10 [23040/60000 (38%)]	Loss: 0.090287
Train Epoch: 10 [23680/60000 (39%)]	Loss: 0.055941
Train Epoch: 10 [24320/60000 (41%)]	Loss: 0.194731
Train Epoch: 10 [24960/60000 (42%)]	Loss: 0.149105
Train Epoch: 10 [25600/60000 (43%)]	Loss: 0.188525
Train Epoch: 10 [26240/60000 (44%)]	Loss: 0.148017
Train Epoch: 10 [26880/60000 (45%)]	Loss: 0.076412
Train Epoch: 10 [27520/60000 (46%)]	Loss: 0.275039
Train Epoch: 10 [28160/60000 (47%)]	Loss: 0.322424
Train Epoch: 10 [28800/60000 (48%)]	Loss: 0.083034
Train Epoch: 10 [29440/60000 (49%)]	Loss: 0.111913
Train Epoch: 10 [30080/60000 (50%)]	Loss: 0.133902
Train Epoch: 10 [30720/60000 (51%)]	Loss: 0.046419
Train Epoch: 10 [31360/60000 (52%)]	Loss: 0.160453
Train Epoch: 10 [32000/60000 (53%)]	Loss: 0.140852
Train Epoch: 10 [32640/60000 (5